In [4]:
def extract_tags(text):
    # coding:utf8
    import jieba
    import pymysql
    import jieba.analyse
    import re

    jieba.load_userdict("C:/Users/Student/Desktop/tags_set0602.txt")
    jieba.analyse.set_stop_words("C:/Users/Student/Desktop/stopword0601.txt")
    pattern_digi = re.compile(r'原PO|原po|原Po|PO|po|Po|\'\'|[0-9]{1,2}|[0-9]+((\.[0-9])+|(0|9|個月內|幼童|成人|男|女|人|號|\.|連戰|連勝|連拜|連敗|年級|度|歲|歲少女|歲兒童|兆|千億|百億|十億|億|千萬|百萬|十萬|萬|千|百|十|元|局|場|次|下|頁|多年|年|年前|年後|日|日前|日後|周|周前|周後|週|週前|週後|月|月前|個月前|月後|個月後|天|時|點|點半|分|分鐘|秒|秒鐘|公分|公尺|公里|死|傷|公斤|公克|%)+)')
    result={}
    # --使用JIEBA：關鍵詞提取
    kw = jieba.analyse.extract_tags(text, topK=20, withWeight=False, allowPOS=())

    # 以正則做最後的清洗，強制刪除無意義的詞語和空值
    for tags_split in kw:
        if re.fullmatch(pattern_digi,tags_split):
            kw.remove(tags_split)
        if kw.__contains__(''):
            kw.remove('')

    # 將清洗完的項目去重
    tags_setted = list(set(kw))

    # 將去重完的tags塞成字串，準備放入SQL
    tags_final = ''
    for sets in tags_setted:
        tags_final = tags_final + sets + '、'

    # 刪除每段tags最後面多出來的"、"
    tags_final = tags_final[:-1]
    result["tags"]=tags_final
    result = classification(result)
    return result

def classification(result):
    import numpy as np
    x_test= [result['tags'].strip("、")]
    print(x_test)

    file_path= 'C:/Users/Student/Desktop/IIIEDU-BDSE10-ML-DL-with-Python/feature_names1.txt'
    file_open = open(file_path,'r',encoding='utf8')
    file_text = file_open.read()
    file_set= set(eval(file_text))
    
    import pickle
    model_path= 'C:/Users/Student/Desktop/IIIEDU-BDSE10-ML-DL-with-Python/clf.pickle'
    model = pickle.load(open(model_path,"rb"))
    
    vocab_path= 'C:/Users/Student/Desktop/IIIEDU-BDSE10-ML-DL-with-Python/vocabulary.txt'
    vocab_open = open(vocab_path,'r',encoding='utf8')
    vocab_text = vocab_open.read()
    vocab = eval(vocab_text)

    
    from sklearn.feature_extraction.text import CountVectorizer   
    count_v1= CountVectorizer(vocabulary=vocab)
    x_test = count_v1.transform(x_test)       
    
    probability = model.predict_proba(x_test)[0]    
    probability = list(map(lambda x: str(int(x*100))+'%',probability))
    y_pred = model.predict(x_test)[0]
    
    classindex = ['國際','娛樂','政治','生活','社會','經濟','運動']
    result_class = classindex[y_pred]
    result['prob']=probability
    result['class']=classindex
    result['result_class']=result_class
    return result

# text='印度今天對蘋果、杏仁等28項美國輸印商品加徵關稅，報復華府當局最近結束對印度的優惠貿易待遇。印度財政部中央間接稅暨關稅稅收管理委員會（Central Board of Indirect Taxes and Customs）表示，要對杏仁、蘋果和核桃等產品加徵關稅。中央間接稅暨關稅稅收管理委員會在公告中表示，印度將對「原產於美國或自美國出口的28項特定產品加徵報復性關稅」。這份清單原本列出29項產品，但最終把豐年蝦從清單中剔除。印度是加州杏仁和華盛頓蘋果的第2大市場。儘管華府戮力加強與印度間的關係來抗衡中國，美國總統川普和印度總理莫迪（Narendra Modi）也雙雙主張彼此關係良好，但美印兩國貿易關係仍陷入緊張。川普決定打擊讓美國出現巨大貿易逆差的國家，華府當局去年宣布要對進口的鋼鋁商品加徵關稅，而且拒絕給予印度豁免。印度回應表示會對美國一系列商品加徵進口關稅，但多次延後加徵關稅，希望藉由談判來處理棘手的貿易議題。然而，川普這個月決定終止對印度的貿易優惠待遇，促使印度做出加徵關稅的決定。'
text='高雄市長韓國瑜今(16)下午突然現身桃園頭寮，向前總統蔣經國陵寢表達追思。韓表示，明天是自己生日，所以今天高雄事情忙一段落，就坐高鐵北上來祭拜經國先生，這是他心中的一個心願。他指出，經國先生過世32年來一直都是台灣人最懷念的總統，這兩天他特別想要來祭拜一下。 韓國瑜今天下午輕車簡從搭著計程車到桃園頭寮經國紀念館，一下車就受到熱情民眾歡迎簇擁，接著他由國民黨第2至第6選區的立委參選人陳學聖、呂玉玲、吳志揚、魯明哲、萬美玲陪同到蔣經國陵寢致意，現場民眾聽說明天是他生日，還高唱「生日快樂歌」祝韓生日快樂。韓國瑜說，他認為蔣經國總統是一個很奇怪複合體，「是包青天加胡雪巖」，不但操守好，執政時政府風氣非常清廉有效，有點像包青天；而且蔣特別會搞經濟建設，十大建設把台灣經濟往上衝，像胡雪巖；大家特別懷念蔣經國總統，他不管任何時候，永遠願意跟基層民眾、庶民在一起，傾聽民意。雖然這麼多年來大家對蔣的評價看法都不一樣，但他始終覺得蔣經國總統有四大特點，操守好、官員有效率清廉、衝經濟、傾聽民意。 韓國瑜表示，他今天來謁陵是發自內心來向經國先生表達尊重之意，並不是來報告選總統一事。他說，回想這二、三十年台灣的政治、經濟還有社會，很多地方要向經國先生學習。 韓國瑜表示，這幾年台灣民眾真的過得太辛苦，下一代2020領導者有責任、有義務，把台灣從經濟困難泥淖之中拉出來，讓台灣安全，中華民國國運強盛，人民能過好日子。很多施政過去沒有做，以後會更困難，經國先生在30多年前可以想到這些事真的很了不起，建議大家都來經國紀念館看一看。'
print(extract_tags(text))

['一段落、經濟、清廉、高雄市長、包青天、傾聽、經國先生、立委、台灣、生日、操守、陵寢、桃園、總統、紀念館、頭寮、蔣經國、經國、祭拜、韓國瑜']
{'tags': '一段落、經濟、清廉、高雄市長、包青天、傾聽、經國先生、立委、台灣、生日、操守、陵寢、桃園、總統、紀念館、頭寮、蔣經國、經國、祭拜、韓國瑜', 'prob': ['0%', '0%', '100%', '0%', '0%', '0%', '0%'], 'class': ['國際', '娛樂', '政治', '生活', '社會', '經濟', '運動'], 'result_class': '政治'}
